In [60]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [61]:
!cd PonyGE2/ && git pull

Already up to date.


In [62]:
#pip installs

%pip install numpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\hsabi\appdata\local\programs\python\python311\lib\site-packages (1.24.1)




[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
%pip install sewar



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%%capture captured
import shutil
import os

experiment = "regression"

shutil.copy(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt
os.remove(f"./PonyGE2/src/fitness/{experiment}.py")



Start:	 2023-05-04 19:34:43.549463 

DATASET_NAME: B1H


Traceback (most recent call last):
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\utilities\algorithm\NSGA2.py", line 291, in __init__
    self.n_objectives = params['FITNESS_FUNCTION'].num_obj
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'regression' object has no attribute 'num_obj'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\ponyge.py", line 31, in <module>
    mane()
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\ponyge.py", line 24, in mane
    individuals = params['SEARCH_LOOP']()
                  ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\algorithm\search_loop.py", line 32, in search_loop
    get_stats(individuals)
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\stats\stats.py", line 64, in get_stats
    get_moo_stats(individuals, end)
  File

In [6]:
import numpy as np
from sewar.full_ref import mse
from openpyxl import load_workbook

captured_str = str(captured).split('\n')
for line in captured_str:
    if "Phenotype: " in line:
        phenotype = line.replace("Phenotype: ", "")
    elif "DATASET_NAME" in line:
        dataset_name = line.replace("DATASET_NAME: ", "")

dataset_path = './datasets/data.xlsx'



In [7]:
def extremeVal(reference, val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

def load_data():
    wb = load_workbook(dataset_path)
    dataset_name = ["CARSALES", "LAKEERIE", "GAS", "Electricity",
                    "PIGS", "POLLUTION", "REDWINE", "SUNSPOT", "Nordic", "B1H"]
    dataset = {}
    for name in dataset_name:

        ws = wb[name]
        columns = [ws["A"], ws["B"], ws["C"],
                   ws["D"], ws["E"], ws["F"], ws["G"]]

        sheet = [[] for _ in range(len(columns[0]))]

        for c in columns:
            for index, item in enumerate(c):
                sheet[index].append(item.value)

        sheet = np.array(sheet)
        dataset[name] = {"raw": sheet}

    for name, _ in dataset.items():
        sheet = dataset[name]["raw"]

        dataset[name]["series"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["arima"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["mlp"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["svr"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["rbf"] = {"treino": [], "teste": [], "validacao": []}

        for row in sheet[1:]:
            split_type = row[2]
            if row[4] == None:
                continue
            key = None
            if split_type == "Teste":
                key = "teste"
            elif split_type == "Validacao":
                key = "validacao"
            elif split_type == "Treinamento":
                key = "treino"
            else:
                continue

            dataset[name]["series"][key].append(float(row[1]))
            dataset[name]["arima"][key] .append(float(row[3]))
            dataset[name]["mlp"][key]   .append(float(row[4]))
            dataset[name]["svr"][key]   .append(float(row[5]))
            dataset[name]["rbf"][key]   .append(float(row[6]))

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["series"][type_data] = np.array(
                dataset[name]["series"][type_data])
            dataset[name]["arima"][type_data] = np.array(
                dataset[name]["arima"][type_data])
            dataset[name]["mlp"][type_data] = np.array(
                dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = np.array(
                dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = np.array(
                dataset[name]["rbf"][type_data])

        fixExtreme = np.vectorize(extremeVal)

    for name in dataset.keys():
        reference = np.max(dataset[name]["series"]["treino"])

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["mlp"][type_data] = fixExtreme(
                reference, dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = fixExtreme(
                reference, dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = fixExtreme(
                reference, dataset[name]["rbf"][type_data])
            dataset[name]["arima"][type_data] = fixExtreme(
                reference, dataset[name]["arima"][type_data])

    return dataset


def build_model(phenotype):
    model = {"linear": {}, "nlinear": {}}

    linear, nlinear = phenotype.split(';')

    weight, linear_model = linear.split(':')
    model["linear"][linear_model] = float(weight)

    nlinear_parts = nlinear.split(',')

    for i in range(len(nlinear_parts)):
        weight, nlinear_model = nlinear_parts[i].split(':')
        model["nlinear"][nlinear_model] = float(weight)

    return model


def predict_mse(dataset, phenotype, series_name):
    model = build_model(phenotype)
    name = list(model["linear"].keys())[0]
    predict = np.zeros(len(dataset[dataset_name][name][series_name]))

    for key, val in model["linear"].items():
        predict += dataset[dataset_name][key][series_name] * val

    for key, val in model["nlinear"].items():

        predict += dataset[dataset_name][key][series_name] * val

    return mse(dataset[dataset_name]["series"][series_name], predict)


In [8]:
dataset = load_data()
print(f'Dataset: {dataset_name}')
print(f'Fenótipo: {phenotype}')
print('Fitness treino:', f"{predict_mse(dataset, phenotype, 'treino'):.3e}")
print('Fitness teste:', f"{predict_mse(dataset, phenotype, 'teste'):.3e}")
print('Fitness validacao:', f"{predict_mse(dataset, phenotype, 'validacao'):.3e}")


Dataset: B1H


NameError: name 'phenotype' is not defined